<a href="https://colab.research.google.com/github/anaslva/Compilador/blob/master/trab_transformada_hough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Alunos: Ana Carolina da Silva e Gustavo Baroni Bruder

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
from pathlib import Path
from zipfile import ZipFile

PARAM_1 = 20
PARAM_2 = 31

# Diretórios
DATA_DIR = "data"
OUTPUT_DIR = "final_iris"

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)


def segmentar_iris(img):
    cinza = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    borrada = cv2.medianBlur(cinza, 5)
    binarizada = cv2.threshold(borrada, 100, 255, cv2.THRESH_BINARY)[1]

    circles = None
    min_r, max_r = 80, 200

    while circles is None or circles.size == 0:
        circles = cv2.HoughCircles(binarizada, cv2.HOUGH_GRADIENT, dp=2, minDist=2000,
                                   param1=PARAM_1, param2=PARAM_2,
                                   minRadius=min_r, maxRadius=max_r)
        max_r += 10

    circles = np.uint16(np.around(circles))
    mascara = np.zeros_like(img)

    for x, y, r in circles[0]:
        cv2.circle(mascara, (x, y), r, (255, 255, 255), -1)

    return cv2.bitwise_and(img, mascara)


def remover_pupila(img):
    cinza = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    borrada = cv2.medianBlur(cinza, 5)
    binarizada = cv2.threshold(borrada, 30, 255, cv2.THRESH_BINARY)[1]

    circles = cv2.HoughCircles(binarizada, cv2.HOUGH_GRADIENT, dp=2, minDist=500,
                                param1=PARAM_1, param2=PARAM_2,
                                minRadius=20, maxRadius=50)

    if circles is None:
        return img

    circles = np.uint16(np.around(circles))
    mascara = np.full_like(img, 255)

    for x, y, r in circles[0]:
        cv2.circle(mascara, (x, y), r, (0, 0, 0), -1)

    return cv2.bitwise_and(img, mascara)


def processar_imagens():
    imagens = [f for f in os.listdir(DATA_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if not imagens:
        print(f"Nenhuma imagem encontrada na pasta '{DATA_DIR}'")
        return

    for nome_arquivo in imagens:
        caminho = os.path.join(DATA_DIR, nome_arquivo)
        img = cv2.imread(caminho)

        iris = segmentar_iris(img)
        iris_sem_pupila = remover_pupila(iris)

        saida = os.path.join(OUTPUT_DIR, nome_arquivo)
        cv2.imwrite(saida, iris_sem_pupila[:, :, ::-1])


def visualizar_resultados():
    arquivos = os.listdir(OUTPUT_DIR)
    plt.figure(figsize=(15, len(arquivos) * 3))

    for i, nome in enumerate(arquivos):
        caminho = os.path.join(OUTPUT_DIR, nome)
        img = cv2.imread(caminho)
        if img is None:
            continue
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        plt.subplot(len(arquivos), 1, i + 1)
        plt.imshow(img_rgb)
        plt.title(nome)
        plt.axis("off")

    plt.tight_layout()
    plt.show()


def compactar_resultado():
    with ZipFile(ZIP_FILE, "w") as zipf:
        for pasta in [DATA_DIR, OUTPUT_DIR]:
            for root, _, files in os.walk(pasta):
                for file in files:
                    caminho_completo = os.path.join(root, file)
                    zipf.write(caminho_completo)


if __name__ == "__main__":
    print("#####Processando imagens#####")
    processar_imagens()

    print("#####Exibindo resultados#####")
    visualizar_resultados()



#####Processando imagens#####


KeyboardInterrupt: 